In [1]:
import numpy as np

def write_var_dym(file_out, tvect, x, y, mask, data, verbose=True):
    """
    Writes data to a DYM file.

    Parameters
    ----------
    file_out : str
        The name of the DYM file.
    tvect : numpy.ndarray
        The date vector in decimal format.
    x : numpy.ndarray
        The vector of longitudes.
    y : numpy.ndarray
        The vector of latitudes.
    mask : numpy.ndarray
        The land mask - a matrix of integers 0 - 2, with
        0 for land, 1 for the sea with only one layer (nearshore),
        2 for the sea with two layers, and 3 for the sea with
        three pelagic layers through the depth column.
    data : numpy.ndarray
        The 3D array of data to be written in the DYM file.
    verbose : bool, optional
        Flag controlling the verbosity of the function.
        Default is True.

    Returns
    -------
    None
    """
    if verbose:
        print(f"Writing file {file_out}...")
    
    # Initialization
    Idfunc = 0
    nlon = len(x)
    nlat = len(y)
    nlevel = len(tvect)
    xlon = np.tile(x, (nlat, 1))
    ylat = np.repeat(y, nlon).reshape(nlat, nlon)[:, ::-1]
    
    # Writing
    with open(file_out, 'wb') as con:
        con.write(b'DYM2')
        con.write(Idfunc.to_bytes(4, byteorder='little'))
        con.write(np.nanmin(data).astype('float32').tobytes())
        con.write(np.nanmax(data).astype('float32').tobytes())
        con.write(nlon.to_bytes(4, byteorder='little'))
        con.write(nlat.to_bytes(4, byteorder='little'))
        con.write(len(tvect).to_bytes(4, byteorder='little'))
        con.write(tvect[0].astype('float32').tobytes())
        con.write(tvect[-1].astype('float32').tobytes())
        
        for i in range(nlat):
            con.write(xlon[i, :].astype('float32').tobytes())
        
        for i in range(nlat):
            con.write(ylat[i, :].astype('float32').tobytes())
        
        con.write(tvect.astype('float32').tobytes())
        
        for i in range(nlat):
            con.write(mask[i, :].astype('int32').tobytes())  # Not recognized by C and Java
        
        # Control for NaN values
        data = np.where(np.isnan(data), -999, data)
        if len(data.shape) == 3 and data.shape[0] != nlevel:
            print("WARNING: number of matrices is not equal to nlevel!")
            data[:nlevel, :, :] = data[0:nlevel, :, :]
        
        for ti in range(nlevel):
            dat = np.apply_along_axis(np.flip, 0, data[ti, :, :]) if len(data.shape) == 3 else np.flip(data, axis=0)
            for i in range(nlat):
                con.write(dat[i, :].astype('float32').tobytes())
    
    print(f"File {file_out} written.")

# Example usage
# write_var_dym("sst.dym", tt, x, y, mask, var)

In [20]:
file_out="./coucou.txt"
tvect=np.arange(np.datetime64("2000-01-01"),np.datetime64("2000-01-11"))
x=np.arange(0.0, 360.0, 1.0) # lon
y=np.arange(0.0, 180.0, 1.0) # lat
data=np.zeros((len(tvect), len(y), len(x)))
mask=np.full((len(y), len(x)), 3)

In [21]:
write_var_dym(
    file_out=file_out,
    tvect=tvect,
    x=x,
    y=y,
    mask=mask, data=data
)

Writing file ./coucou.txt...
File ./coucou.txt written.
